In [1]:
#basic package
import json
import tqdm
import os
import numpy as np
import glob
import pandas as pd
import sys
import shutil
import time
import datetime as dt
import itertools
import time
import re
from collections import Counter
from collections import defaultdict
import operator
import math
import pickle
from collections import Counter

#modelling
import statsmodels
import statsmodels.api as sm
import statsmodels.formula.api as smf

#plot
import matplotlib
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf, acf, pacf

from matplotlib.collections import LineCollection
from matplotlib.colors import BoundaryNorm
import seaborn as sns

c:\users\camil\anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
from UTILS import perc_element_dico
import config_mobility as config

we restrict the time series to one value per 60 seconds 
we compute the complexity variables each 30 minutes 
each variables includes the values of at least the last 120.00 minutes (i.e. are using 120.00 values)


In [3]:
path_extracted_data = config.path_extracted_data
path_initial_data = config.path_initial_data
id_run = config.id_run
path_save_ = os.path.join(path_extracted_data, 'visual', 'egg','ACF-PACF_allhour')

# Download data

In [19]:
#egg shift
df_shift = pd.read_csv(os.path.join(path_save_,id_run+'df_acf_pacf_session_summary.csv'), sep=';')
display(df_shift.head(3))
#keep only with 60mn
df_shift = df_shift[df_shift['mn']==60]
dico_hen_nbrunsign = dict(df_shift.groupby(['HenID'])['pacf_confintdifference'].agg(lambda x: sum([i<=0.1 for i in x])))
#remove not strong significicant pacf
df_shift = df_shift[df_shift['pacf_confintdifference']>0.1]

df_shift['pacf_value'] = df_shift['pacf_value'].map(lambda x: round(x))
li_val = df_shift['pacf_value'].unique()
#extract variables
fct2apply = {'pacf_variance':lambda x: round(np.var(x),3),
            'pacf_dico':lambda x: perc_element_dico(x)}
df_shift = df_shift.groupby(['HenID'])['pacf_value'].agg(fct2apply).reset_index()
for v in li_val:
    df_shift['pacf_perc_'+str(int(v))] = df_shift['pacf_dico'].map(lambda x: round(x.get(float(v),0.0),3))
df_shift['perc_before24'] = df_shift['pacf_dico'].map(lambda x: sum([v for k,v in x.items() if k<24]))
df_shift['perc_after24'] = df_shift['pacf_dico'].map(lambda x: sum([v for k,v in x.items() if k>24]))
df_shift['nbr_unsign'] = df_shift['HenID'].map(lambda x: dico_hen_nbrunsign[x])
#df_shift.drop(columns=['pacf_dico'], inplace=True)
print(df_shift.shape)
df_shift.head(3)

,HenID,mn,pacf_confintdifference,pacf_value,session
0,hen_1,20,0.114295,23.666667,10A
1,hen_1,30,0.172965,24.500000,10A
2,hen_1,60,0.352229,24.000000,10A


c:\users\camil\anaconda3\lib\site-packages\ipykernel_launcher.py:15: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  from ipykernel import kernelapp as app


(120, 25)


,HenID,pacf_variance,pacf_dico,pacf_perc_24,pacf_perc_23,pacf_perc_29,pacf_perc_25,pacf_perc_27,pacf_perc_20,pacf_perc_21,...,pacf_perc_28,pacf_perc_16,pacf_perc_19,pacf_perc_13,pacf_perc_11,pacf_perc_14,pacf_perc_17,perc_before24,perc_after24,nbr_unsign
0,hen_1,0.400,"{24: 60.0, 25: 20.0, 23: 20.0}",60.000,20.000,0.0,20.000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000,0.0,0.0,20.000,20.000,1.0
1,hen_10,0.000,{25: 100.0},0.000,0.000,0.0,100.000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.000,100.000,1.0
2,hen_100,21.918,"{24: 14.286, 11: 14.286, 23: 28.571, 25: 42.857}",14.286,28.571,0.0,42.857,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,14.286,0.0,0.0,42.857,42.857,1.0


In [23]:
#egg variable
df_egg = pd.read_csv(os.path.join(path_initial_data,'Mo_eggs_matched_with_ID.csv'), sep=';', parse_dates=['Date'])
print(df_egg.shape)
#df_egg = df_egg[df_egg['Strength']!='na']
df_egg['Date']= df_egg['Date'].replace(dt.datetime(2017,9,15), dt.datetime(2017,8,15)) #TODO: ASK YAMENAH CORRECT?
df_egg = df_egg.replace('na', np.nan)
df_egg['Strength'] = df_egg['Strength'].astype(float)
df_egg['Mass'] = df_egg['Mass'].astype(float)
df_egg['Width'] = df_egg['Width'].astype(float)
df_egg.rename(columns={'Date':'egg_level', 'HenID_Christina':'HenID'}, inplace=True)
df_egg['HenID'] = df_egg['HenID'].map(lambda x: 'hen_'+str(x))
df_egg['is_LSL'] = df_egg['hybrid'].map(lambda x: float(x=='LSL'))
print(df_egg[df_egg['Strength'].isnull()].shape, df_egg[df_egg['Mass'].isnull()].shape, df_egg[df_egg['Width'].isnull()].shape)
print(df_egg.shape)
df_egg.head(3)

(465, 20)
(9, 21) (1, 21) (1, 21)
(465, 21)


,Pen,egg_level,CollDay,Location,Mass,Strength,Width,Color,PenColour,HenInPen,...,hybrid,HenID_Legband,HenID,HenID_Filipe,timepoint,FileName,severity,gap,date_mob,is_LSL
0,1,2017-08-07,d4,nest,64.2,47.0,0.31,bbr,1bbr,12,...,LSL,1.12,hen_12,12,11,11A12,2.9,0.0,2017-08-06,1.0
1,1,2017-08-07,d4,nest,72.9,46.0,0.28,mbr,1mbr,18,...,LSL,1.18,hen_18,18,11,11A18,5.5,0.0,2017-08-06,1.0
2,1,2017-08-07,d4,nest,68.1,33.0,0.25,rbr,1rbr,15,...,LSL,1.15,hen_15,15,11,11A15,6.9,1.0,2017-08-06,1.0


In [24]:
#MLPS session metrics
li_path = glob.glob(os.path.join(path_extracted_data,'VF_time_serie_day_*_variables_SIMPLER.csv'))
dico_sess_dfvar = {}
li_zone = ['zone_1', 'zone_2', 'zone_3', 'zone_4', 'zone_5']
for path_ in li_path:
    sessID = path_.split('_')[-3]
    df_ = pd.read_csv(path_, sep=';', index_col=0, parse_dates=['level','FirstTimestamp_zone_1','FirstTimestamp_zone_2',
                                                                'FirstTimestamp_zone_3','FirstTimestamp_zone_4',
                                                                'FirstTimestamp_zone_5'])
    #day 2017-08-07 egg quality must be linked to the MLPs variable of the day 2017-08-06 with possible bias induce by if the hen 
    #laid the egg at 9h on the 2017-08-06 (or anytime after 8h, the time where the people take the egg). 
    df_['egg_level'] = df_['level'].map(lambda x: x+dt.timedelta(days=1))
    for z in li_zone:
        df_['SampEnt_'+z] = df_['SampEnt_perZone'].map(lambda x: eval(x).get(z,0))
        df_['SampEnt_'+z] = df_['SampEnt_'+z].replace(-0.0,0)
    df_.drop(columns=['level','verification_daily_total_duration','SampEnt_perZone'], inplace=True)    
    #put to have less change among the dependant variable entries
    df_['SampEnt_order2'] = df_['SampEnt_order2'].map(lambda x: x*10000)
    for v in ['Max_duration','duration_zone_1','duration_zone_2','duration_zone_3','duration_zone_4','duration_zone_5']:
        df_[v] = df_[v].map(lambda x: x/60/60)
    df_['Variance_duration'] = df_['Variance_duration'].map(lambda x: x/(60*60)/(60*60))
    li_fts = ['FirstTimestamp_zone_1','FirstTimestamp_zone_2','FirstTimestamp_zone_3','FirstTimestamp_zone_4',
              'FirstTimestamp_zone_5']
    for x in li_fts:
        df_[x+'_h'] = df_[x].map(lambda x: x.hour+x.minute/60+x.second/60/60)
        #lets say if we dont know that they went to 17h
        df_[x+'_h'].fillna(17, inplace=True)
    dico_sess_dfvar[sessID] = df_
print(len(dico_sess_dfvar))
dico_sess_dfvar['10B'].head(3)

22


,HenID,duration_zone_1,duration_zone_2,duration_zone_3,duration_zone_4,duration_zone_5,FirstTimestamp_zone_1,FirstTimestamp_zone_2,FirstTimestamp_zone_3,FirstTimestamp_zone_4,...,SampEnt_zone_1,SampEnt_zone_2,SampEnt_zone_3,SampEnt_zone_4,SampEnt_zone_5,FirstTimestamp_zone_1_h,FirstTimestamp_zone_2_h,FirstTimestamp_zone_3_h,FirstTimestamp_zone_4_h,FirstTimestamp_zone_5_h
0,hen_100,0.012778,4.205833,0.991389,1.286944,8.503056,2017-07-15 10:32:17,2017-07-15 02:21:31,2017-07-15 02:08:22,2017-07-15 02:07:03,...,0.002235,0.018886,0.014852,0.018080,0.022160,10.538056,2.358611,2.139444,2.117500,2.0
1,hen_100,1.045556,1.544167,0.977222,1.473889,9.959167,2017-07-16 09:29:54,2017-07-16 03:49:39,2017-07-16 03:05:41,2017-07-16 02:48:18,...,0.007679,0.010937,0.009991,0.010793,0.012134,9.498333,3.827500,3.094722,2.805000,2.0
2,hen_100,0.102778,1.851111,0.018333,1.223056,11.804722,2017-07-17 10:21:23,2017-07-17 04:30:03,2017-07-17 04:41:58,2017-07-17 04:42:44,...,0.002257,0.008536,0.002235,0.010307,0.010099,10.356389,4.500833,4.699444,4.712222,2.0


In [25]:
df_var11 = pd.concat([dico_sess_dfvar['11A'],dico_sess_dfvar['11B']])
print(df_var11.shape)
df_var11.head(3)

(504, 48)


,HenID,duration_zone_1,duration_zone_2,duration_zone_3,duration_zone_4,duration_zone_5,FirstTimestamp_zone_1,FirstTimestamp_zone_2,FirstTimestamp_zone_3,FirstTimestamp_zone_4,...,SampEnt_zone_1,SampEnt_zone_2,SampEnt_zone_3,SampEnt_zone_4,SampEnt_zone_5,FirstTimestamp_zone_1_h,FirstTimestamp_zone_2_h,FirstTimestamp_zone_3_h,FirstTimestamp_zone_4_h,FirstTimestamp_zone_5_h
0,hen_1,0.0,0.000000,0.000000,2.260556,12.739444,NaT,NaT,NaT,2017-08-03 04:30:49,...,0.0,0.000000,0.000000,0.032790,0.032790,17.0,17.000000,17.000000,4.513611,2.0
1,hen_1,0.0,0.026111,0.036667,2.151389,12.785833,NaT,2017-08-04 13:41:51,2017-08-04 13:40:05,2017-08-04 03:24:23,...,0.0,0.002237,0.002235,0.034894,0.032449,17.0,13.697500,13.668056,3.406389,2.0
2,hen_1,0.0,0.026667,0.030833,2.217778,12.724722,NaT,2017-08-05 09:20:14,2017-08-05 09:19:30,2017-08-05 05:01:23,...,0.0,0.002235,0.002242,0.029955,0.027494,17.0,9.337222,9.325000,5.023056,2.0


In [26]:
#TODO: why only 84??

In [27]:
li_hen_mlp = df_var11['HenID'].unique()
li_hen_egg = df_egg['HenID'].unique()
print(len(li_hen_mlp), len(li_hen_egg))

84 106


In [28]:
df_egg = df_egg[df_egg['HenID'].isin(li_hen_mlp)]
print(df_egg.shape)

(345, 21)


# Modelling

## daily ~ MLP metrics daily including only timestamp 11

#### aggregate variable at day level

In [41]:
df = pd.merge(df_egg.filter(['egg_level','HenID','Date','Location','Mass','Strength','Width','hybrid','group','severity',
                             'is_LSL']),
              df_var11, on=['egg_level','HenID'], how='left')
print(df.shape)
df.head(3)

(345, 56)


,egg_level,HenID,Location,Mass,Strength,Width,hybrid,group,severity,is_LSL,...,SampEnt_zone_1,SampEnt_zone_2,SampEnt_zone_3,SampEnt_zone_4,SampEnt_zone_5,FirstTimestamp_zone_1_h,FirstTimestamp_zone_2_h,FirstTimestamp_zone_3_h,FirstTimestamp_zone_4_h,FirstTimestamp_zone_5_h
0,2017-08-07,hen_12,nest,64.2,47.0,0.31,LSL,A,2.9,1.0,...,0.0,0.002283,0.006795,0.034288,0.027683,17.0,9.871944,9.785556,3.980278,2.0
1,2017-08-07,hen_18,nest,72.9,46.0,0.28,LSL,A,5.5,1.0,...,0.0,0.000000,0.000000,0.012291,0.012291,17.0,17.000000,17.000000,2.865278,2.0
2,2017-08-07,hen_6,nest,65.3,53.0,0.27,LSL,A,5.6,1.0,...,0.0,0.004494,0.011394,0.072840,0.061064,17.0,6.333611,6.330000,2.203333,2.0


In [42]:
df = pd.merge(df, df_shift, on=['HenID'], how='left')
print(df.shape)
df.head(3)

(345, 80)


,egg_level,HenID,Location,Mass,Strength,Width,hybrid,group,severity,is_LSL,...,pacf_perc_28,pacf_perc_16,pacf_perc_19,pacf_perc_13,pacf_perc_11,pacf_perc_14,pacf_perc_17,perc_before24,perc_after24,nbr_unsign
0,2017-08-07,hen_12,nest,64.2,47.0,0.31,LSL,A,2.9,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.000,0.000,1.0
1,2017-08-07,hen_18,nest,72.9,46.0,0.28,LSL,A,5.5,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,44.444,0.000,2.0
2,2017-08-07,hen_6,nest,65.3,53.0,0.27,LSL,A,5.6,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,27.273,0.0


In [43]:
df.columns

Index(['egg_level', 'HenID', 'Location', 'Mass', 'Strength', 'Width', 'hybrid',
       'group', 'severity', 'is_LSL', 'duration_zone_1', 'duration_zone_2',
       'duration_zone_3', 'duration_zone_4', 'duration_zone_5',
       'FirstTimestamp_zone_1', 'FirstTimestamp_zone_2',
       'FirstTimestamp_zone_3', 'FirstTimestamp_zone_4',
       'FirstTimestamp_zone_5', 'Total_number_zone',
       'RunSampEnt_onLastTsOfEachLevel', 'RunDistEnt_onLastTsOfEachLevel',
       'RunEnt_onLastTsOfEachLevel_nbr_value',
       'RunEnt_onLastTsOfEachLevel_ts_value',
       'RunSampEnt_onLastTsOfEachLevel_1', 'RunSampEnt_onLastTsOfEachLevel_2',
       'RunSampEnt_onLastTsOfEachLevel_3', 'RunSampEnt_onLastTsOfEachLevel_4',
       'RunSampEnt_onLastTsOfEachLevel_5', 'Max_duration_zones',
       'Max_duration', 'Min_duration', 'Median_duration', 'Average_duration',
       'Variance_duration', 'dico_zone_duration', 'Total_number_transition',
       'distribution_entropy', 'nbr_bouts_zone_5', 'nbr_bouts_zone_

#### choosing variables

In [26]:
#sorted(df_var11['egg_level'].unique())

In [27]:
#df_egg[df_egg['egg_level']=='2017-08-13']

In [28]:
#df[df['duration_zone_2'].isnull()]['HenID'].value_counts()

In [29]:
#df[df['duration_zone_2'].isnull()]['egg_level'].value_counts()

In [30]:
#df_var11[(df_var11['HenID']=='hen_81')&(df_var11['egg_level']=='hen_81')]

In [31]:
#df.isna().sum()
#duration_zone_1!!!!

In [44]:
#for now: remove those values
print(df.shape)
df = df[~((df['duration_zone_2'].isnull())&(df['duration_zone_4'].isnull()))]
df.shape

(345, 80)


(238, 80)

In [45]:
#fill nan
li = ['SampEnt_order2', 'SampEnt_zone_1','SampEnt_zone_2', 'SampEnt_zone_3', 'SampEnt_zone_4', 'SampEnt_zone_5',
          'RunSampEnt_onLastTsOfEachLevel', 'RunDistEnt_onLastTsOfEachLevel',
          'RunSampEnt_onLastTsOfEachLevel_1', 'RunSampEnt_onLastTsOfEachLevel_2',
          'RunSampEnt_onLastTsOfEachLevel_3', 'RunSampEnt_onLastTsOfEachLevel_4',
          'RunSampEnt_onLastTsOfEachLevel_5',
          'distribution_entropy']
df[li] = df[li].fillna(0.0)

In [47]:
x = 'Mass' 

li_x = ['Location', 'Mass', 'Strength', 'Width'] 

li_hen = ['is_LSL', 'severity']

li_mlp = ['duration_zone_1', 'duration_zone_2','duration_zone_3', 'duration_zone_4', 'duration_zone_5',
           'FirstTimestamp_zone_1_h', 'FirstTimestamp_zone_2_h',
           'FirstTimestamp_zone_3_h', 'FirstTimestamp_zone_4_h',
           'FirstTimestamp_zone_5_h', 'Total_number_zone','Max_duration_zones',
           'Max_duration', 'Min_duration', 'Median_duration', 'Average_duration',
           'Variance_duration', 'dico_zone_duration', 'Total_number_transition',
           'nbr_bouts_zone_5', 'nbr_bouts_zone_4',
           'nbr_bouts_zone_3', 'nbr_bouts_zone_2', 'nbr_bouts_zone_1',
           'Max_duration_zone_4']

li_cpx = ['SampEnt_order2', 'SampEnt_zone_1','SampEnt_zone_2', 'SampEnt_zone_3', 'SampEnt_zone_4', 'SampEnt_zone_5',
          'RunSampEnt_onLastTsOfEachLevel', 'RunDistEnt_onLastTsOfEachLevel',
          'RunSampEnt_onLastTsOfEachLevel_1', 'RunSampEnt_onLastTsOfEachLevel_2',
          'RunSampEnt_onLastTsOfEachLevel_3', 'RunSampEnt_onLastTsOfEachLevel_4',
          'RunSampEnt_onLastTsOfEachLevel_5',
          'distribution_entropy']     

li_pacf = ['pacf_variance', 'pacf_perc_24', 'pacf_perc_23', 'pacf_perc_26',
           'pacf_perc_25', 'pacf_perc_29', 'pacf_perc_22', 'pacf_perc_27',
           'pacf_perc_28', 'pacf_perc_30', 'pacf_perc_15', 'pacf_perc_18',
           'pacf_perc_21', 'pacf_perc_19', 'perc_before24', 'perc_after24','nbr_unsign']
li_pacf_before24 = ['pacf_perc_23', 'pacf_perc_22', 'pacf_perc_15', 'pacf_perc_18',
           'pacf_perc_21', 'pacf_perc_19']


li_var = list(set(['is_LSL','duration_zone_1','duration_zone_2','duration_zone_5','duration_zone_4',
          'Total_number_transition', 'severity','Variance_duration','distribution_entropy','SampEnt_order2','SampEnt_zone_4',
          'Max_duration_zone_4','FirstTimestamp_zone_4_h']+li_pacf_before24))

#### mixed effect model

In [ ]:
li_var = ['duration_zone_1','duration_zone_2','duration_zone_5','duration_zone_4',
          'Total_number_transition', 'severity','Variance_duration','distribution_entropy','SampEnt_order2','SampEnt_zone_4',
          'Max_duration_zone_4','pacf_perc_24','perc_after24','perc_before24','FirstTimestamp_zone_4_h','is_LSL',
         'Mass', 'Strength','nbr_unsign']
g = sns.PairGrid(df[li_var])
g.map(plt.scatter);
plt.savefig(os.path.join(path_save_,id_run+'_scatterplot_var.png'), format='png', dpi=300)
#Total_number_transition, duration_zone_2 , SampEnt_zone_4up --> perc_after24 up and perc_before24 down
#Max_duration_zone_4, duration_zone_4 up --> erc_after24 down and perc_before24 up
#??Variance_duration?? fucked up  --'SampEnt_order2','SampEnt_zone_4' down

In [ ]:
li_var = ['perc_after24','perc_before24','SampEnt_zone_4','duration_zone_2','duration_zone_4','Max_duration_zone_4',
          'Total_number_transition']
g = sns.PairGrid(df[li_var])
g.map(plt.scatter);
plt.savefig(os.path.join(path_save_,id_run+'_scatterplot_var.png'), format='png', dpi=300)
#Total_number_transition, duration_zone_2 , SampEnt_zone_4up --> perc_after24 up and perc_before24 down
#Max_duration_zone_4, duration_zone_4 up --> perc_after24 down and perc_before24 up
#??Variance_duration?? fuck up up  --'SampEnt_order2','SampEnt_zone_4' down

In [ ]:
# let's use the lmplot function within seaborn
sns.lmplot(x = "perc_before24", y = "Mass", data = df)

In [ ]:
li_var = list(set(['duration_zone_1','duration_zone_2','duration_zone_5','duration_zone_4',
          'Total_number_transition', 'severity','distribution_entropy','SampEnt_order2','SampEnt_zone_4',
          'Max_duration_zone_4','perc_before24','perc_after24','FirstTimestamp_zone_4_h','is_LSL','nbr_unsign']))
mm = sm.MixedLM(endog=df['Mass'], exog=df[li_var], groups=df['HenID'])
res = mm.fit(method='lbfgs') #method='lbfgs','cg'
res.summary()

####  GLM

In [ ]:
#https://pypi.org/project/statsmodels-dq/
#e.g.: https://docs.w3cub.com/statsmodels/examples/notebooks/generated/glm_formula/

#perc_before24, perc_after24
formula = x+' ~ '+'+'.join(li_var)
min_sample_Size = 50 + 8*len(li_var)
df_mod = df[~df[x].isnull()][li_var+[x,'HenID']].copy()
display(df_mod.head(3))
print(min_sample_Size)
if min_sample_Size>=df_mod.shape[0]:
    print('WARNING: reduce the dependant var or augment your number of observation')
    sys.exit()
    
mod1 = smf.glm(formula=formula, data=df_mod).fit() #, family=sm.families.Binomial()
mod1.summary()
#--> mass seems to increase with higher percentgae of shift 22

#### DL (if more data)

https://medium.com/@urvashilluniya/why-data-normalization-is-necessary-for-machine-learning-models-681b65a05029
---------------dependant var multi class categorical variable
use softmax as activation function and sparse-categorical cross entropy as loss function
model = keras.Sequential([
 keras.layers.Dense(64, activation=tf.nn.relu,                  
 input_shape=(y_train.shape[1],)),
 keras.layers.Dense(64, activation=tf.nn.relu),
 keras.layers.Dense(#nbr_categories, activation=  'softmax')
 ])

model.compile(optimizer=tf.train.AdamOptimizer(),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
              
-----------continuous var ,mse as loss relu as activation finish with 1
model = keras.Sequential([keras.layers.Dense(64, activation=tf.nn.relu,
                                             input_shape=(y_train.shape[1],)),
                                             keras.layers.Dense(64, activation=tf.nn.relu),
                                             keras.layers.Dense(1, activation=  'relu')])
display(model.summary())
model.compile(optimizer=tf.train.AdamOptimizer(),
              loss='mean_squared_error',
              metrics=['accuracy'])

from sklearn import preprocessing
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.python.data import Dataset
import keras
from keras.utils import to_categorical
from keras import models
from keras import layers


x = 'Mass' 
li_pacf_before24 = ['pacf_perc_23', 'pacf_perc_22', 'pacf_perc_15', 'pacf_perc_18',
           'pacf_perc_21', 'pacf_perc_19']
li_var = list(set(['is_LSL','duration_zone_1','duration_zone_2','duration_zone_5','duration_zone_4',
          'Total_number_transition', 'severity','Variance_duration','distribution_entropy','SampEnt_order2','SampEnt_zone_4',
          'Max_duration_zone_4','pacf_perc_23', 'pacf_perc_22']))

df_mod = df[~df[x].isnull()][li_var+[x]].copy()

#Select predictors
y = df_mod[li_var]
#Target variable 
x = df_mod[x]
#Split data into train and test 
y_train, y_test, x_train, x_test = train_test_split(y, x , train_size = 0.8, random_state =  90)
'''As y variable is multi class categorical variable, hence using softmax as activation function and sparse-categorical cross entropy as loss function.'''
model = keras.Sequential([keras.layers.Dense(64, activation=tf.nn.relu,
                                             input_shape=(y_train.shape[1],)),
                                             keras.layers.Dense(64, activation=tf.nn.relu),
                                             keras.layers.Dense(1, activation=  'relu')])
display(model.summary())
model.compile(optimizer=tf.train.AdamOptimizer(),
              loss='mean_squared_error',
              metrics=['accuracy'])
history1 = model.fit(y_train, x_train,
                     epochs= 200, batch_size = 10,
                     validation_data = (y_test, x_test))

x = 'Location' 
li_pacf_before24 = ['pacf_perc_23', 'pacf_perc_22', 'pacf_perc_15', 'pacf_perc_18',
           'pacf_perc_21', 'pacf_perc_19']
li_var = list(set(['is_LSL','duration_zone_1','duration_zone_2','duration_zone_5','duration_zone_4',
          'Total_number_transition', 'severity','Variance_duration','distribution_entropy','SampEnt_order2','SampEnt_zone_4',
          'Max_duration_zone_4','pacf_perc_23', 'pacf_perc_22']))

df_mod = df[~df[x].isnull()][li_var+[x]].copy()

#Select predictors
y = df_mod[li_var]

#Target variable 
x = df_mod[x]

#Split data into train and test 
y_train, y_test, x_train, x_test = train_test_split(y, x , train_size = 0.8, random_state =  90)

model = keras.Sequential([
 keras.layers.Dense(64, activation=tf.nn.relu,                  
 input_shape=(y_train.shape[1],)),
 keras.layers.Dense(64, activation=tf.nn.relu),
 keras.layers.Dense(len(df['Location'].unique()), activation=  'softmax')
 ])

model.compile(optimizer=tf.train.AdamOptimizer(),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
history1 = model.fit(y_train, x_train,
                     epochs= 2000, batch_size = 10,
                     validation_data = (y_test, x_test))

## hen level ~ MLP metrics at hen level including all timestamp

# to think about